# Generating function of the number of occurrences of periodic subword in a periodic word

This is a Sagemath Notebook implementing the effective computation of the generating function

$$ f_{w, v}(x, y) = \sum_{m, r \geq 0} \operatorname{occ}(w^m, v^r) x^m y^r. $$

Here, $\operatorname{occ}(w, v)$ is the number of occurrences of the subword $v$ in the word $w$.

In Theorem 4.13 of my article "Maximal number of subword occurrences in a word", available [here](https://arxiv.org/abs/2406.02971), we show that $f_{w, v}(x, y)$ is always a rational fraction of $x, y$, and the computation is effective. This Sagemath Notebook contains a function that does this computation for **binary words**.

## Utilities for computing subword occurrences

In [1]:
from itertools import combinations, product
from random import randrange

bcache = {} # cache for binomial coefficients

def binomial(n: int, k: int) -> int:
    '''
    Compute the binomial coefficient, with memoization
    '''
    if n < 0 or k < 0 or n < k:
        return 0
    if k == 0 or k == n:
        return 1
    if (n, k) in bcache:
        return bcache[(n, k)]
    res = binomial(n - 1, k - 1) + binomial(n - 1, k)
    bcache[(n, k)] = res
    return res

def subword_count_list(wl: tuple[int], swl: tuple[int]):
    '''
    Computes the number of occurrences of the subword represented by swl in the word represented by wl.

    We assume that the words represented by wl and swl start with the same letter.
    
    INPUT:
    - `wl`: tuple of run lengths of the word
    - `swl`: tuple of run lengths of the subword

    OUTPUT:
    The number of occurrences of the subword in the word
    '''
    if not swl:
        return 1
    if not wl or len(wl) < len(swl):
        return 0
    accu : int = 0
    seg : int = swl[0]
    newswl : list[int] = list(swl)
    for i in range(seg - 1): # number of symbols that we consume (but not all!)
        newswl[0] -= 1
        mult : int = binomial(wl[0], i + 1)
        if mult > 0:
            accu += subword_count_list(wl[2:], tuple(newswl)) * mult
    # case of consuming all symbols in the first segment
    if wl[0] >= swl[0]:
        accu += subword_count_list(wl[1:], swl[1:]) * binomial(wl[0], swl[0])
    # case of no removal
    accu += subword_count_list(wl[2:], swl)
    return accu

def subword_count(word : str, subword : str) -> int:
    '''
    Computes the number of occurrences of `subword` in the `word`.

    We assume that both strings are binary, containing only 0 and 1.
    '''
    if len(subword) == 0:
        return 1
    if len(word) == 0:
        return 0
    def get_wlist(w: str):
        '''
        This function computes the list of run lengths of the word `w`.
        '''
        wl = [1]
        for i in range(len(w) - 1):
            if w[i] == w[i + 1]:
                wl[-1] += 1
            else:
                wl.append(1)
        return tuple(wl)
    wl : list[int] = get_wlist(word)
    swl : list[int] = get_wlist(subword)
    if word[0] != subword[0]: # not sharing the same first letter
        wl = wl[1:]
    return subword_count_list(wl, swl)

In [2]:
# a sanity check

print(subword_count("011100010111010100010110111001001110", "0110011010110110")); # record for n = 36

1047330


## Implementation of Theorem 4.13

In [3]:
x, y = var('x y')
RatRing = FractionField(PolynomialRing(QQ, [x, y]))

def simple_occ(w: str, v: str, s: int, t: int) -> int:
    '''
    This function computes a_{w, v}^{s, t)(1)
    '''
    # special case: when v is of length 1
    if len(v) == 1 and (s != t or w[s] != v[0]):
        return 0
    # the positions s and t do not match the first and last letters of v
    if w[s] != v[0] or w[t] != v[-1]:
        return 0
    # not enough letters
    if s == t and len(v) != 1:
        return 0
    if s + len(v) - 1 > t:
        return 0
    return subword_count(w[s + 1 : t], v[1:-1])

def single_occ_gf(w: str, v: str, s: int, t: int):
    '''
    This function compute g_{w, v}^{s, t)(x)
    '''
    accu = RatRing(simple_occ(w, v, s, t)) # constant term
    vlen = len(v)
    for lsigma in range(2, vlen + 1):
        lsum = 0
        for sigma in Compositions(vlen, length=lsigma): # sum over partitions of clusters
            swlist = list(sigma)
            partialsum = [0]
            for e in swlist:
                partialsum.append(partialsum[-1] + e)
            vsegs = [v[partialsum[i]:partialsum[i + 1]] for i in range(len(swlist))]
            lprod = 1
            for i in range(1, len(vsegs) - 1):
                lprod *= subword_count(w, vsegs[i])
            # head and tail segments
            lprod *= sum(simple_occ(w, vsegs[0], s, tp) for tp in range(s, len(w)))
            lprod *= sum(simple_occ(w, vsegs[-1], sp, t) for sp in range(t + 1))
            lsum += lprod
        accu += RatRing(lsum * x ^ (lsigma - 1) * (1 - x) ^ (1 - lsigma))
    return accu

In [4]:
def period_occ_gf(w: str, v: str):
    '''
    This function compute f_{w, v}(x, y)
    '''
    vlen, wlen = len(v), len(w)
    single_gf = [[single_occ_gf(w, v, s, t) for t in range(wlen)] for s in range(wlen)]
    # first term
    constant_term = [y * sum(single_gf[s][t] / (1 - x) for s in range(wlen)) for t in range(wlen)]
    m = [[0] * wlen for _ in range(wlen)]
    for t in range(wlen):
        for tprime in range(wlen):
            # second term
            m[t][tprime] = constant_term[t] * x
            # third term
            m[t][tprime] += y * sum(single_gf[s][t] for s in range(tprime + 1, wlen))
        # LHS
        m[t][t] -= 1
    constant_vec = vector(constant_term)
    sysM = Matrix(m)
    sol = sysM.solve_right(constant_vec)
    return (1 / (1 - x) - x * sum(e for e in sol) / (1 - x)).simplify_full()

## Some examples

In [5]:
period_occ_gf('01', '01').show()

-(x - 1)/(x^2 - x*y - 2*x + 1)

In [6]:
period_occ_gf('0011', '01').show()

-(x - 1)/(x^2 - 4*x*y - 2*x + 1)

In [7]:
period_occ_gf('000111', '0011').show()

-(x^3 - 3*x^2 + 3*x - 1)/(x^4 - 4*x^3 + 6*x^2 - 9*(4*x^3 + 4*x^2 + x)*y - 4*x + 1)

In [ ]:
period_occ_gf('0001100111', '0011').show()